In [ ]:
%cd ..

In [ ]:
from damply import dirs
from pathlib import Path
from tqdm import tqdm
from joblib import Parallel, delayed

from imgtools.io.writers.nifti_writer import NIFTIWriter, NiftiWriterIOError

from readii.process.images.crop import crop_and_resize_image_and_mask
from readii.negative_controls_refactor.manager import NegativeControlManager

from readii_2_roqc.utils.loaders import load_dataset_config, load_image_and_mask
from readii_2_roqc.utils.metadata import get_masked_image_metadata, insert_SampleID
from readii_2_roqc.utils.settings import get_readii_settings, get_resize_string

import pandas as pd
import itertools
import logging
import SimpleITK as sitk

In [ ]:
dataset = "NSCLC-Radiomics_test"
overwrite = True
seed = 10

# Set up logging

In [ ]:
logger = logging.getLogger(__name__)
logging.basicConfig(filename = dirs.LOGS / f"{dataset}_make_negative_controls.log", encoding='utf-8', level=logging.DEBUG)

# Load dataset config

In [ ]:
if dataset is None:
    message = "Dataset name must be provided."
    logger.error(message)
    raise ValueError(message)

dataset_config, dataset_name, full_dataset_name = load_dataset_config(dataset)
# logger.info(f"Creating negative controls for dataset: {dataset_name}")

# Load dataset index

In [ ]:
images_dir_path = dirs.PROCDATA / full_dataset_name / 'images'

dataset_index = pd.read_csv(images_dir_path / f'mit_{dataset_name}' / f'mit_{dataset_name}_index-simple.csv')
dataset_index = insert_SampleID(dataset_index)

# Filter the index by R2R configuration file

In [ ]:
masked_image_index = get_masked_image_metadata(dataset_index, dataset_config)

# Check for output(s) existence

In [ ]:
regions, permutations, crop, resize = get_readii_settings(dataset_config)

readii_image_dir = images_dir_path / f'readii_{dataset_name}'

if crop != "" and resize != []:
    readii_index_filepath = readii_image_dir / f"{crop}_{get_resize_string(resize)}" / f'readii_{dataset_name}_index.csv'
else:
    readii_index_filepath = readii_image_dir.glob(f"original_*/readii_{dataset_name}_index.csv").__next__()

In [ ]:
if readii_index_filepath.exists() and not overwrite:
    regions, permutations, crop, resize = get_readii_settings(dataset_config)
    # Load in readii index and check:
    # 1. if all negative controls requested have been extracted
    # 2. for all of the patients
    readii_index = pd.read_csv(readii_index_filepath)

    # Get list of patients that have already been processed and what has been requested based on the dataset index
    processed_samples = set(readii_index['PatientID'].to_list())
    requested_samples = set(dataset_index['SampleID'].to_list())


    readii_settings = ['Permutation', 'Region', 'crop', 'Resize']
    if not set(readii_index.columns).issuperset(readii_settings):
        print("Not all READII settings satisfied in existing output. Re-running negative control generation.")
    
    else:
        processed_image_types = {itype for itype in readii_index[readii_settings].itertuples(index=False, name=None)}
        requested_image_types = {itype for itype in itertools.product(permutations,
                                                                      regions,
                                                                      [crop],
                                                                      [get_resize_string(resize)])}
        if requested_image_types.issubset(processed_image_types) and requested_samples.issubset(processed_samples):
            print("All requested negative controls have already been generated for these samples or are listed in the readii index as if they have been. Set overwrite to true if you want to re-process these.")

# Negative control generator function that can be run in parallel

In [ ]:
def negative_control_generator(sample_id:str,
                               image:sitk.Image,
                               mask:sitk.Image,
                               regions, 
                               permutations, 
                               crop, 
                               resize,
                               image_meta_id,
                               mask_meta_id,
                               nifti_writer:NIFTIWriter,
                               seed = 10
                               ):
    negative_control_image_paths = []

    # Set up negative control manager with settings from config
    manager = NegativeControlManager.from_strings(
        negative_control_types=permutations,
        region_types=regions,
        random_seed=seed
    )
    # Process and save negative control images
    for proc_image, permutation, region in manager.apply(image, mask):
        # apply crop and resize
        if crop != "" and resize != []:
            proc_image, proc_mask = crop_and_resize_image_and_mask(proc_image, 
                                                                   mask, 
                                                                   crop_method = crop, 
                                                                   resize_dimension = resize)
        # save out negative controls
        try:
            # The capitalized arguments are here on purpose to manipulate the order of the columns in the index file
            out_path = nifti_writer.save(
                            proc_image,
                            ImageID = image_meta_id,
                            MaskID = mask_meta_id,
                            Permutation="original",
                            Region="full",
                            Resize=get_resize_string(resize),
                            SampleID=sample_id,
                            crop=crop
                        )
        except NiftiWriterIOError:
            message = f"{permutation} {region} negative control file already exists for {sample_id}. If you wish to overwrite, set overwrite to True."
            logger.debug(message)
        
        negative_control_image_paths.append(out_path)

    return negative_control_image_paths

In [ ]:
def image_preprocessor(dataset_config:dict, 
                        image_path:Path, 
                        mask_path:Path, 
                        images_dir_path:Path, 
                        output_dir:Path, 
                        sample_id:str = None, 
                        mask_image_id:str = None, 
                        overwrite:bool = False):
    if overwrite:
        existing_file_mode = 'OVERWRITE'
        overwrite_index = True
    else:
        existing_file_mode = 'SKIP'
        overwrite_index = False

    regions, permutations, crop, resize = get_readii_settings(dataset_config)
    resize_string = get_resize_string(resize)

    # Get sample metadata from path if not provided
    if sample_id is None:
        sample_id = Path(image_path).parts[0]
    if mask_image_id is None:
        mask_image_id = Path(mask_path).name.removesuffix('.nii.gz')
    
    # Get beginning of the path to the nifti images dir
    mit_images_dir = images_dir_path / f'mit_{dataset_name}'
    # load in the nifti image and mask files, flattened to 3D and aligned with each other
    image, mask = load_image_and_mask(mit_images_dir / image_path, mit_images_dir / mask_path)
    # get image modality for file writer
    image_modality = dataset_config['MIT']['MODALITIES']['image']
    # make image identifier string for file writer
    image_meta_id = f"{image_path.parent.name}"
    # make mask identifier string for file writer
    mask_meta_id = f"{mask_path.parent.name}_{mask_image_id.replace(' ', "_")}"

    # Set up the readii subdirectory for the image being processed, specifically the crop and resize level
    if crop == '' and resize == []:
        # get the original image size to use for output directory, without the slice count
        image_size_string = get_resize_string(image.GetSize()[0:2])
        # make the slice index an n since different images have different slice counts
        crop_setting_string = f'original_{image_size_string}_n'
    else:
        crop_setting_string = f'{crop}_{resize_string}'

    # Set up writer for saving out the negative controls and index file
    nifti_writer = NIFTIWriter(
            root_directory = output_dir / crop_setting_string,
            filename_format = f"{image_path.parent}/{mask_meta_id}/{image_modality}" + "_{Permutation}_{Region}.nii.gz",
            create_dirs = True,
            existing_file_mode = existing_file_mode,
            sanitize_filenames = True,
            index_filename = f"readii_{dataset_name}_index.csv",
            overwrite_index = overwrite_index
        )
    
    readii_image_paths = []
    # Process crop and resize of original image if needed, and save
    if crop != '' and resize != []:
        crop_image, crop_mask = crop_and_resize_image_and_mask(image, 
                                                               mask, 
                                                               crop_method = crop, 
                                                               resize_dimension = resize)
        # save out cropped image
        try:
            # The capitalized arguments are here on purpose to manipulate the order of the columns in the index file
            out_path = nifti_writer.save(
                            crop_image,
                            ImageID = image_meta_id,
                            MaskID = mask_meta_id,
                            Permutation="original",
                            Region="full",
                            Resize=resize_string,
                            SampleID=sample_id,
                            crop=crop
                        )
        except NiftiWriterIOError:
            message = f"{crop} {resize_string} original image file already exists for {sample_id}. If you wish to overwrite, set overwrite to True."
            logger.debug(message)
        
        readii_image_paths.append(out_path)
    # end original image processing

    if permutations != [] and regions != []:
        negative_control_image_paths = negative_control_generator(sample_id=sample_id,
                                                                  image = image,
                                                                  mask = mask,
                                                                  regions = regions,
                                                                  permutations = permutations,
                                                                  crop = crop,
                                                                  resize = resize,
                                                                  image_meta_id = image_meta_id,
                                                                  mask_meta_id = mask_meta_id,
                                                                  nifti_writer = nifti_writer)
        readii_image_paths.append(negative_control_image_paths)
    
    return readii_image_paths


In [ ]:
from readii_2_roqc.utils.metadata import make_edges_df

edges_index = make_edges_df(masked_image_index, dataset_config['MIT']['MODALITIES']['image'], dataset_config['MIT']['MODALITIES']['mask'])
parallel = True
n_jobs = 4

if parallel:
    # Use joblib to parallelize negative control generation
    readii_image_paths = Parallel(n_jobs=n_jobs)(
                            delayed(image_preprocessor)(
                                dataset_config=dataset_config, 
                                image_path=Path(data_row.filepath_image), 
                                mask_path=Path(data_row.filepath_mask), 
                                images_dir_path=images_dir_path, 
                                output_dir=readii_image_dir,
                                sample_id=data_row.SampleID_image,
                                mask_image_id=data_row.ImageID_mask, 
                                overwrite=overwrite
                            )
                            for _, data_row in tqdm(
                                edges_index.iterrows(),
                                desc="Generating negative controls for each image-mask pair...",
                                total=len(edges_index)
                            )
                        )
else:
    readii_image_paths = [image_preprocessor(dataset_config=dataset_config, 
                                            image_path=Path(data_row.filepath_image), 
                                            mask_path=Path(data_row.filepath_mask), 
                                            images_dir_path=images_dir_path, 
                                            output_dir=readii_image_dir,
                                            sample_id=data_row.SampleID_image,
                                            mask_image_id=data_row.ImageID_mask, 
                                            overwrite=overwrite
                                            ) for _, data_row in tqdm(edges_index.iterrows(),
                                                                        desc="Generating negative controls for each image-mask pair...",
                                                                        total=len(edges_index))]
 
